In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
# Creating Spark Session
sp = SparkSession.builder.appName("Customer").getOrCreate()

In [ ]:
df_cc = sp.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_CREDIT_CARD") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

df_cust = sp.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_CUSTOMER") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

df_br = sp.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_BRANCH") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [ ]:
df_cc.show()

In [ ]:
df_join = df_cc.join(df_cust, df_cc.CUST_SSN == df_cust.SSN, 'inner').select(col('SSN'), col('CUST_ZIP'), col('TIMEID'), \
      col('TRANSACTION_ID'), col('TRANSACTION_VALUE'))

In [ ]:
df_join.show()

In [ ]:
df_join.createOrReplaceTempView('CC_CUST')

In [ ]:
# data = sp.sql('select * from CC_CUST where CUST_ZIP == "53066" and substr(TIMEID,1,4) == "2018" and substr(TIMEID,5,2)=="06"')
# data.show(100)

In [ ]:
def trans_value(zip,yr,mm):
    print("inside the function", zip, yr, mm)
    data = sp.sql('select distinct * from CC_CUST where CUST_ZIP == "{}" and substr(TIMEID,1,4) == "{}" \
        and substr(TIMEID,5,2) == "{}" ORDER BY substr(TIMEID,7,2) DESC'.format(zip,yr,mm))
    data.show()


zip = str(input("Enter 5-digit zipcode: "))
yr = str(input("Enter year: "))
mm = str(input("Enter month: "))
trans_value(zip,yr,mm)

Functional requirements 2.1 (2)

In [ ]:
# df_cc.select('transaction_type','transaction_value').groupby('transaction_type').agg(count('transaction_type'), \
    # sum('transaction_value')).show()

In [ ]:
df_cc.createOrReplaceTempView('cdw_cc')

In [ ]:
data1 = sp.sql('SELECT transaction_type, COUNT(*) as trans_count, \
                round(SUM(transaction_value),2) as trans_val \
                FROM cdw_cc \
                GROUP BY transaction_type')

In [ ]:
data1.show()

Functional requirements 2.1 (3)

In [ ]:
data_br = df_cc.join(df_br, df_br.BRANCH_CODE == df_cc.BRANCH_CODE, 'inner').select(df_br['branch_code'].alias('BC1'), \
            col('branch_name'), col('branch_state'), col('transaction_value'))
data_br.show()

In [ ]:
data_br.createOrReplaceTempView('data_br_view')

In [ ]:
def trans_by_branch(state):
    data1 = sp.sql("SELECT bc1, COUNT(bc1), \
                round(SUM(transaction_value),2) \
                FROM data_br_view \
                WHERE BRANCH_STATE == '{}' \
                GROUP BY bc1 \
                ORDER BY bc1".format(state))
    data1.show()

state = str(input("Enter State : "))
trans_by_branch(state)